In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

# データの読み込み

In [ ]:
train_data = pd.read_csv('../data/preprocessed_data/train_clean.csv')
test_data = pd.read_csv('../data/preprocessed_data/test_clean.csv')

# 追加前処理

In [ ]:
# 'id'と'price'を分離
train_ids = train_data['id']
train_y = train_data['price']
test_ids = test_data['id']

train_data.drop(['id', 'price'], axis=1, inplace=True)
test_data.drop(['id'], axis=1, inplace=True)

In [ ]:
# 数値でないカテゴリ変数のみをOne-Hotエンコード
encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='error')
X_train_encoded = encoder.fit_transform(train_data.select_dtypes(exclude=['int64', 'float64']))
X_test_encoded = encoder.transform(test_data.select_dtypes(exclude=['int64', 'float64']))

In [ ]:
# 数値変数を結合
X_train = pd.concat([pd.DataFrame(X_train_encoded), train_data.select_dtypes(include=['int64', 'float64'])], axis=1)
X_test = pd.concat([pd.DataFrame(X_test_encoded), test_data.select_dtypes(include=['int64', 'float64'])], axis=1)

# モデル定義

In [ ]:
# モデルの定義
def build_model(hp):
    inputs = Input(shape=(X_train.shape[1]))
    x = inputs
    
    for i in range(hp.Int('num_layers', 1, 5)):
        units = hp.Int('units_' + str(i), min_value=32, max_value=512, step=32)
        x = Dense(units, activation='relu')(x)
        x = Dropout(hp.Float('dropout_' + str(i), min_value=0, max_value=0.5, step=0.1))(x)
        outputs = Dense(1)(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=Adam(hp.Choice('learning_rate', [0.01, 0.005, 0.001])), loss=tf.keras.losses.MeanAbsolutePercentageError())
        return model

In [ ]:
# ランダムサーチの実施
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=3,
    directory='random_search03'
)

# データの分割

In [ ]:
# データを訓練用と検証用に分割
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, train_y, test_size=0.2, random_state=42)

# チューニングの実行（バッチサイズの調整）
tuner.search(X_train_split, y_train_split, epochs=30, validation_data=(X_val_split, y_val_split), batch_size=512)

# 学習

In [ ]:
# 最適なモデルの取得
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# 最適なモデルで予測
predictions = best_model.predict(X_test).flatten()

# 提出ファイルの出力

In [ ]:
# 結果をCSVファイルとして出力
result = pd.DataFrame({'id': test_ids, 'price': predictions})
result.to_csv('../submit-file/20230816_01_submit.csv', index=False, header=False)